In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

from art.defences.trainer import AdversarialTrainerAWPPyTorch

import time

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            # 'TPR': '',
            # 'FNR': '',
            # 'TNR': '',
            # 'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/Defense/AWP/AWP_PGD.csv", mode='a', index=False)

In [3]:
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, f1_score,recall_score, roc_auc_score
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def calculate_performance_metrics(x_test_adv, y_test, classifier, model_name, attack_name, epsilon):

    start_time = time.time()

    preds = np.argmax(classifier.predict(x_test_adv), axis=-1)
    # preds = classifier.predict(x_test_adv)

    y_hat = preds
    # y_test_arg = np.argmax(y_test, axis=1)
    y_test_arg = y_test


    accuracy = accuracy_score(y_test_arg, y_hat)
    precision_macro = precision_score(y_test_arg, y_hat, average='macro')
    # precision_micro = precision_score(y_test_arg, y_hat, average='micro')
    precision_weighted = precision_score(y_test_arg, y_hat, average='weighted')
    f1_macro = f1_score(y_test_arg, y_hat, average='macro')
    # f1_micro = f1_score(y_test_arg, y_hat, average='micro')
    f1_weighted = f1_score(y_test_arg, y_hat, average='weighted')
    recall_macro = recall_score(y_test_arg, y_hat, average='macro')
    # recall_micro = recall_score(y_test_arg, y_hat, average='micro')
    recall_weighted = recall_score(y_test_arg, y_hat, average='weighted')


    print(f"epsilon = {epsilon}")

    print(f"Accuracy: {accuracy}")
    # print(f"Precision(micro,macro,weighted): {precision_micro},{precision_macro},{precision_weighted}")
    # print(f"F1(micro,macro,weighted: {f1_micro},{f1_macro},{f1_weighted}")
    # print(f"Recall(micro,macro,weighted: {recall_micro},{recall_macro},{recall_weighted}")
    print(f"Precision(macro,weighted): {precision_macro},{precision_weighted}")
    print(f"F1(macro,weighted: {f1_macro},{f1_weighted}")
    print(f"Recall(macro,weighted: {recall_macro},{recall_weighted}")


    
    metrics = {
        "model" : model_name,
        "attack_model" : attack_name,
        "epsilon" : epsilon,
        "Accuracy": accuracy,
        # "Precision-micro": precision_micro,
        "Precision-macro": precision_macro,
        "Precision-weighted": precision_weighted,
        # "Recall_micro": recall_micro,
        "Recall_macro": recall_macro,
        "Recall_weighted": recall_weighted,
        
        # "F1-micro": f1_micro,
        "F1-macro": f1_macro,
        "F1-weighted": f1_weighted,

    }
    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_csv("/home/jovyan/Defense/AWP/AWP_PGD.csv", mode='a', index=False, header=False)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time: {elapsed_time:.2f} second\n\n")

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [5]:
x_test = np.load('/home/jovyan/Wustl_iiot/x_test.npy')
x_train = np.load('/home/jovyan/Wustl_iiot/x_train.npy')
x_val = np.load('/home/jovyan/Wustl_iiot/x_val.npy')
y_test = np.load('/home/jovyan/Wustl_iiot/y_test.npy')
y_train = np.load('/home/jovyan/Wustl_iiot/y_train.npy')
y_val = np.load('/home/jovyan/Wustl_iiot/y_val.npy')


# train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
# train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

# val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
# val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [6]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [7]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [8]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/Wustl_iiot/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [9]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_val = x_val.astype(np.float32)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [11]:
from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent

FGSM = FastGradientMethod(estimator=classifier, eps=0.3)
BIM = BasicIterativeMethod(estimator=classifier, eps=0.3, max_iter=3, verbose = False)
PGD = ProjectedGradientDescent(estimator=classifier, eps=0.3, max_iter=3, verbose = False)

In [12]:
FGSM_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, FGSM, "TRADES", 0.1, 6.0, 0)
BIM_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, BIM, "TRADES", 0.1, 6.0, 0)
PGD_Trainer = AdversarialTrainerAWPPyTorch(classifier, classifier, PGD, "TRADES", 0.1, 6.0, 0)

In [13]:
# start_time = time.time()
# FGSM_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
#                      validation_data=(x_val, y_val))
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Time: {elapsed_time:.2f} second\n\n")

In [14]:
# start_time = time.time()
# BIM_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
#                      validation_data=(x_val, y_val))
# end_time = time.time()
# elapsed_time = end_time - start_time
# print(f"Time: {elapsed_time:.2f} second\n\n")

In [15]:
start_time = time.time()
PGD_Trainer.fit(x_train, y_train, epochs=10, batch_size=128,
                     validation_data=(x_val, y_val))
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time: {elapsed_time:.2f} second\n\n")

Adversarial Training AWP with TRADES - Epochs: 100%|██████████| 20/20 [2:44:20<00:00, 493.04s/it]  

Time: 9860.77 second




In [16]:
# calculate_performance_metrics(x_test, y_test, BIM_Trainer, 'BIM_Trainer', 'baseline', '0')
# calculate_performance_metrics(x_test, y_test, FGSM_Trainer, 'FGSM_Trainer', 'baseline', '0')
calculate_performance_metrics(x_test, y_test, PGD_Trainer, 'PGD_Trainer', 'baseline', '0')

epsilon = 0
Accuracy: 0.9978932201899033
Precision(macro,weighted): 0.7900447921283311,0.9977402873059991
F1(macro,weighted: 0.72262789120227,0.9976698851339068
Recall(macro,weighted: 0.6770070948469007,0.9978932201899033
Time: 1.48 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, BIM_Trainer, 'BIM_Trainer', 'BIM', epsilon)

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, FGSM_Trainer, 'FGSM_Trainer', 'FGSM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Wustl_iiot/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, PGD_Trainer, 'PGD_Trainer', 'PGD', epsilon)

/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.01
Accuracy: 0.9971937525392351
Precision(macro,weighted): 0.7879351476693495,0.9970735860143937
F1(macro,weighted: 0.7074262540226514,0.9968377734088358
Recall(macro,weighted: 0.6567401045556386,0.9971937525392351
Time: 1.52 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.1
Accuracy: 0.022227992946685485
Precision(macro,weighted): 0.048397275747738736,0.10639376996395174
F1(macro,weighted: 0.03931802824405716,0.015507188490391028
Recall(macro,weighted: 0.09199261605651333,0.022227992946685485
Time: 1.46 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.2
Accuracy: 0.0027769284574434665
Precision(macro,weighted): 0.008595731528950473,0.03661785247426284
F1(macro,weighted: 0.001233575221661219,0.004794211761107005
Recall(macro,weighted: 0.0068016259480634484,0.0027769284574434665
Time: 1.47 second




/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jovyan/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


epsilon = 0.3
Accuracy: 0.0025633185761016614
Precision(macro,weighted): 0.008710045956277113,0.03654684979362086
F1(macro,weighted: 0.0011441730826117114,0.004790630975279053
Recall(macro,weighted: 0.0006123055597139339,0.0025633185761016614
Time: 1.52 second




In [19]:
torch.save(PGD_Trainer, "/home/jovyan/Defense/AWP/AWP_PDG.pt")